In [33]:
import json
from stmpy import Machine, Driver
import logging
import random
import paho.mqtt.client as mqtt
from threading import Thread
import ipywidgets as widgets
from IPython.display import display

broker, port = "ipsen.no", 1883
id = "19e0b6bf-4d0f-43c2-ae93-90e42e62ba70"

In [34]:
class Charger:
    def __init__(self):
        self.electricity = 0
        self.button_disconnect = widgets.Button(description="Charger disconnected")
        self.button_disconnect.on_click(self.on_button_disconnect)
        self.button_fully_charged = widgets.Button(description="Car fully charged")
        self.button_fully_charged.on_click(self.on_button_fully_charged)
        self.button_down = widgets.Button(description="Charger out of order")
        self.button_down.on_click(self.on_button_charger_out_of_order)
        self.button_repair = widgets.Button(description="Charger repaired")
        self.button_repair.on_click(self.on_button_charger_repaired)
        display(self.button_disconnect, self.button_fully_charged, self.button_down, self.button_repair)

    def on_button_disconnect(self, b):
        self.stm.send('disconnected')

    def on_button_fully_charged(self, b):
        self.stm.send('fully_charged')

    def on_button_charger_out_of_order(self, b):
        self.stm.send('down')
    
    def on_button_charger_repaired(self, b):
        self.stm.send('repaired')

    def msg_cloud(self, type, message):
        print(message)
        msg = {"msg": message}
        json_msg = json.dumps(msg)
        self.mqtt_client.publish(f"charger/{id}{type}", json_msg)

    def start_measure_electricity(self):
        self.electricity = 0

    def end_measure_electricity(self):
        self.electricity = random.randint(10,150)
        self.msg_cloud("charge", self.electricity)
 
 
s_idle = {
    'name': 'idle',
    'entry': 'msg_cloud("/status","idle")'
}

s_active = {
    'name': 'active',
    'entry': 'msg_cloud("/status","charging");start_measure_electricity()',
    'exit': 'end_measure_electricity();'

}

s_down = {
    'name': 'down',
    'entry': 'msg_cloud("/status","out_of_order")',
    'start': 'msg_cloud("/status","out_of_order")',
    'exit' : 'msg_cloud("/status","charger_repaired")'
}

t_init = {'source': 'initial',
        'target': 'idle'
      }

t_start = {'trigger': 'start',
          'source': 'idle',
          'target': 'active'
          }

t_end = {'trigger': 'end',
          'source': 'active',
         'target': 'idle'
         }

t_fully_charged = {'trigger': 'fully_charged',
          'source': 'active',
         'target': 'idle',
         'effect': 'msg_cloud("", "car_fully_charged")'
         }

t_disconnected = {'trigger': 'disconnected',
          'source': 'active',
         'target': 'idle',
         'effect': 'msg_cloud("", "charger_disconnected")'
         }


t_idle_down = {'trigger': 'down',
                'source': 'idle',
               'target': 'down'
               }

t_idle_self_down = {'trigger': 'self_down',
                'source': 'idle',
               'target': 'down'
               }

t_active_down = {'trigger': 'down',
                'source': 'active',
                 'target': 'down'
                 }

t_active_self_down = {'trigger': 'self_down',
                'source': 'active',
                 'target': 'down'
                 }


t_repaired = {'trigger': 'repaired', 
              'source': 'down',
              'target': 'idle'
              }


In [35]:
class MQTT_Client:
    def __init__(self):
        self.id = id
        self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1)
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
 
 
    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))
 
    def on_message(self, client, userdata, msg):
        topic = msg.topic
        payload_str = msg.payload.decode("utf-8")
        msg_content = ""
        try:
            msg_content = json.loads(payload_str)['msg']
        except json.JSONDecodeError:
            print("Failed to decode JSON")
        print("incomming_message:", msg_content)
        if (msg_content == "start_charging"):
            self.stm_driver.send("start", "charger_stm")
        elif (msg_content == "stop_charging"):
            self.stm_driver.send("end", "charger_stm")
        elif (msg_content == "charger_down"):
            self.stm_driver.send("down", "charger_stm")
        elif (msg_content == "repair"):
            self.stm_driver.send("repaired", "charger_stm")
        #For debug and simulation purposes, event_disconnected, event_car_charged, event_self_down are implemented with MQTT
        elif (msg_content == "event_disconnected"):
            self.stm_driver.send("disconnected", "charger_stm")
        elif (msg_content == "event_car_charged"):
            self.stm_driver.send("fully_charged", "charger_stm")
        elif (msg_content == "event_self_down"):
            self.stm_driver.send("self_down", "charger_stm")
        else:
            print(topic, msg_content)
 
    def start(self, broker, port):
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(f"cmd/charger/{self.id}/#")   
 
        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()
 

In [36]:
charger = Charger()
charger_machine =  Machine('charger_stm', [t_init, t_start, t_end, t_disconnected, t_fully_charged, t_idle_down, t_active_down, t_idle_self_down, t_active_self_down, t_repaired], charger, [s_idle, s_active, s_down])
charger.stm = charger_machine
 
driver = Driver()
driver.add_machine(charger_machine)
 
myclient = MQTT_Client()
charger.mqtt_client = myclient.client
myclient.stm_driver = driver
 
driver.start()
myclient.start(broker, port)
 
driver.start()

Button(description='Charger disconnected', style=ButtonStyle())

Button(description='Car fully charged', style=ButtonStyle())

Button(description='Charger out of order', style=ButtonStyle())

Button(description='Charger repaired', style=ButtonStyle())

idle
Connecting to ipsen.no:1883


...

on_connect(): Connection Accepted.


Machine charger_stm is in state idle and received event end, but no transition with this event is declared!
Machine charger_stm is in state idle and received event end, but no transition with this event is declared!


incomming_message:incomming_message: stop_charging
 stop_charging


Machine charger_stm is in state active and received event start, but no transition with this event is declared!


incomming_message: start_charging
charging
incomming_message: start_charging
23
out_of_order
